In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets 
    #nltk spacy
    #!python -m spacy download en_core_web_sm
else:
    print('We are in the TIRA sandbox.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 KB 1.1 MB/s eta 0:00:00
  Attempting uninstall: tira
    Found existing installation: tira 0.0.132
    Uninstalling tira-0.0.132:
      Successfully uninstalled tira-0.0.132


In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1', 'com.github.terrierteam:terrier-prf:-SNAPSHOT'])
    from jnius import autoclass

Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.8, helper_version=0.0.8, no_download=True


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# generate custom stopword list
nltk.download('stopwords')
nltk_stopwords = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
combined_stopwords = set.union(nltk_stopwords, spacy_stopwords, sklearn_stopwords)

!rm -Rf /tmp/index
file_path = "custom_stopwords.txt"

with open(file_path, 'w+') as file:
    for element in combined_stopwords:
        file.write(element+ "\n")

pt.set_property('stopwords.filename','./custom_stopwords.txt')

ModuleNotFoundError: No module named 'nltk'

In [ ]:
data = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [ ]:
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

In [ ]:
print('Build index:')
indexer = pt.IterDictIndexer("/tmp/index", overwrite = True, blocks = True,meta = {'docno':100, 'text': 20480}, stemmer = 'PorterStemmer')
!rm -Rf /tmp/index
index_ref = indexer.index(data.get_corpus_iter())

print('Done. Index is created')

In [ ]:
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True)
pl2 = pt.BatchRetrieve(index, wmodel="PL2", verbose=True)

In [ ]:
#Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index) 

#Pipeline
pipe = (bm25 % 100) >> bo1 >> pl2

In [ ]:
print('Create run')

run = pipe(topics)

print('Done, run was created')

persist_and_normalize_run(run, 'bm25-custom-stopwords')